In [1]:
import pandas as pd
import numpy as np
import pickle
from scipy.stats import pearsonr
import math
import matplotlib.pyplot as plt

from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow as tf

import sys
import time

sys.path.append("/home/bis/2021_SJH_detectability/Detectability")
from utils import *

In [2]:
df_kb = pd.read_csv('../data/massIVE-KB/df_kb_filter.csv')
df_uni = pd.read_csv('../data/uniprot/df_uni.csv')

In [ ]:
df_ided_peptide = pd.read_csv('data/massIVE-KB/df_kb_strip.csv')
df_ided_protein = pd.read_csv('data/uniprot/df_uni_ided.csv')

pep_tree = tree_from_pep(df_ided_peptide.PEPTIDE.unique())  # from utils

# Make Detect_pep, unDetect_pep version==whole
  - spectral cnt >= 2 : detect
  - spectral cnt == 0 : undetect

In [14]:
def pep_from_prot(df_uni, MISS_CLEAVAGE, thres=4):
    peptide = []
    TS_AA = 'KR'

    for protein, pname, cnt, cnt_miss in df_uni[['SEQUENCE', 'PROTEIN', 'SPECTRAL_CNT', 'SPECTRAL_CNT_MISS']].values:
        cnt = list(map(lambda x: int(x[2:]), cnt.split(';')))
        cnt_miss = list(map(lambda x: int(x[2:]), cnt_miss.split(';')))
        
        ts_idx = []  # tryptic site index
        for prot_idx, aa in enumerate(protein):
            if aa in TS_AA:
                ts_idx.append(prot_idx)
        for idx in range(len(ts_idx)):
            n = MISS_CLEAVAGE
            if idx+(n+1) > len(ts_idx)-1:  # peptide making range
                break

            # protein N term
            if (ts_idx[idx]<=(thres - 1)) and (ts_idx[idx] >= len(protein) -1 -(thres - 1)):  # -MNQKLLK- 앞뒤 다 부족한 경우
                en = 'Z' * (thres - ts_idx[idx]) + protein[: ts_idx[idx]+(thres + 1)] + 'Z' * (thres - (len(protein)-1 - ts_idx[idx]))
            elif ts_idx[idx]<=(thres - 1):
                en = 'Z' * (thres - ts_idx[idx]) + protein[: ts_idx[idx]+(thres + 1)]
            elif ts_idx[idx] >= len(protein) -1 -(thres - 1):  # for EAQDRRN : 끝이 부족한 경우
                en = protein[ts_idx[idx]-thres:] + 'Z' * (thres - (len(protein)-1 - ts_idx[idx]))
            else:
                en = protein[ts_idx[idx]-thres: ts_idx[idx]+(thres + 1)]

            # protein C term
            if (ts_idx[idx+n+1] >= len(protein) -1 -(thres - 1)) and (ts_idx[idx+n+1] <= (thres - 1)):
                ec = 'Z' * (thres - ts_idx[idx+n+1]) + protein[: ts_idx[idx+n+1]+(thres + 1)]  + 'Z' * (thres - (len(protein)-1 - ts_idx[idx+n+1]))
            elif ts_idx[idx+n+1] >= len(protein) -1 -(thres - 1):
                ec = protein[ts_idx[idx+n+1] - thres :] + 'Z' * (thres - (len(protein)-1 - ts_idx[idx+n+1]))
            elif ts_idx[idx+n+1] <= (thres - 1):  # for -MRRS : 시작이 부족한 경우
                ec = 'Z' * (thres - ts_idx[idx+n+1]) + protein[: ts_idx[idx+n+1]+(thres + 1)]
            else:
                ec = protein[ts_idx[idx+n+1] - thres : ts_idx[idx+n+1] +(thres + 1)]  # n+1번째 tryptic_site
                    
            # Peptide miss cleavage
            if n != 0:  # miss cleavage 고려하는 경우,
                ei = []
                for i in range(1, n+1):
                    if ts_idx[idx+i] >= len(protein) -1 -(thres - 1):
                        ei_tmp = protein[ts_idx[idx+i] - thres :] + 'Z' * (thres - (len(protein)-1 - ts_idx[idx+i]))
                    elif ts_idx[idx+i] <= (thres - 1):
                        ei_tmp = 'Z' * (thres - ts_idx[idx+i]) + protein[: ts_idx[idx+i]+(thres + 1)]
                    else:
                        ei_tmp = protein[ts_idx[idx+i] - thres : ts_idx[idx+i] +(thres + 1)]
                    ei.append(ei_tmp)
            else:  # miss cleavage 고려안하는 경우
                ei = []
            # peptide
            if ts_idx[idx+n+1] == len(protein)-1:  # protein C term = idx + n + 1 의 tryptic_site 가 단백질의 마지막인 경우
                pep = protein[ts_idx[idx]] + '.' + protein[ts_idx[idx]+1:ts_idx[idx+n+1]+1] + '.Z'
            else:
                pep = protein[ts_idx[idx]] + '.' + protein[ts_idx[idx]+1:ts_idx[idx+n+1]+1] + '.' + protein[ts_idx[idx+n+1]+1]

            peptide.append({pep:[[en], [ec], ei, pname]})
    return peptide

In [15]:
# STRIP_PEPTIDE SPECTRAL_CNT
pep2cnt = dict()
for p, cnt in df_ided_peptide[['PEPTIDE', 'SPECTRAL_CNT']].values:
    if p not in pep2cnt:
        pep2cnt[p] = 0
    pep2cnt[p] += cnt
    
    
def pep_detection_labelling(df_ided_protein=df_ided_protein, pep_tree=pep_tree, pep2cnt=pep2cnt):
    peptide0 = pep_from_prot(df_ided_protein, 0, thres=7)
    peps = [list(dic.keys())[0] for dic in peptide0]
    ens = [list(dic.values())[0][0] for dic in peptide0]
    ecs = [list(dic.values())[0][1] for dic in peptide0]
    eis = [list(dic.values())[0][2] for dic in peptide0]
    pname = [list(dic.values())[0][3] for dic in peptide0]
    zero = pd.DataFrame([[p, n[0], c[0], '-', '-', pn] 
                         for p, n, c, m, pn in zip(peps, ens, ecs, eis, pname)],
                columns=['peptide', 'En', 'Ec', 'E1', 'E2', 'protein'])
    zero['PEP'] = [i.split('.')[1] for i in zero.peptide.values]

    peptide1 = pep_from_prot(df_ided_protein, 1, thres=7)
    peps = [list(dic.keys())[0] for dic in peptide1]
    ens = [list(dic.values())[0][0] for dic in peptide1]
    ecs = [list(dic.values())[0][1] for dic in peptide1]
    eis = [list(dic.values())[0][2] for dic in peptide1]
    pname = [list(dic.values())[0][3] for dic in peptide1]
    one = pd.DataFrame([[p, n[0], c[0], m[0], '-', pn]
                        for p, n, c, m, pn in zip(peps, ens, ecs, eis, pname)],
                columns=['peptide', 'En', 'Ec', 'E1', 'E2', 'protein'])
    one['PEP'] = [i.split('.')[1] for i in one.peptide.values]

    peptide2 = pep_from_prot(df_ided_protein, 2, thres=7)
    peps = [list(dic.keys())[0] for dic in peptide2]
    ens = [list(dic.values())[0][0] for dic in peptide2]
    ecs = [list(dic.values())[0][1] for dic in peptide2]
    eis = [list(dic.values())[0][2] for dic in peptide2]
    pname = [list(dic.values())[0][3] for dic in peptide2]
    two = pd.DataFrame([[p, n[0], c[0], m[0], m[1], pn]
                        for p, n, c, m, pn in zip(peps, ens, ecs, eis, pname)],
                columns=['peptide', 'En', 'Ec', 'E1', 'E2', 'protein'])
    two['PEP'] = [i.split('.')[1] for i in two.peptide.values]

    zero_filter = zero.loc[zero.PEP.apply(lambda x: (len(x)>=7) and (len(x)<=30))]
    one_filter = one.loc[one.PEP.apply(lambda x: (len(x)>=7) and (len(x)<=30))]
    two_filter = two.loc[two.PEP.apply(lambda x: (len(x)>=7) and (len(x)<=30))]

    df_fully = pd.concat([zero_filter, one_filter, two_filter], axis=0).reset_index(drop=True)

    # preprocessing
    amino = list("ARNDCEQGHILKMFPSTWYVZ")
    check_u = [idx for idx, t in enumerate(df_fully.PEP.values) if 'U' in t]
    check_x = [idx for idx, t in enumerate(df_fully.PEP.values) if 'X' in t]
    check_n = [idx for idx, t in enumerate(df_fully.En.values) if (sum([1 for a in t if a not in amino]) >= 1) and t !='-']
    check_c = [idx for idx, t in enumerate(df_fully.Ec.values) if (sum([1 for a in t if a not in amino]) >= 1) and t !='-']
    check_1 = [idx for idx, t in enumerate(df_fully.E1.values) if (sum([1 for a in t if a not in amino]) >= 1) and t !='-']
    check_2 = [idx for idx, t in enumerate(df_fully.E2.values) if (sum([1 for a in t if a not in amino]) >= 1) and t !='-']
    check = list(set(check_n).union(set(check_c)).union(set(check_1)).union(set(check_2)).union(set(check_u)).union(set(check_x)))
    print('del rows(including U, X amino acid) cnt : {:,}'.format(len(check)))
    df_fully = df_fully.drop(check).reset_index(drop=True)

    ided = []
    for fully_pep in df_fully.PEP.values:
        results = [i for i in pep_tree.search_all(fully_pep)]
        if len(results) != 0:
            result_pep = [_[0] for _ in results]
            if fully_pep in result_pep:  # 정확히 똑같은 pep 있는 경우
                if pep2cnt[fully_pep] > 1:
                    ided.append(1)
                else:
                    ided.append(-1)  # SC1인 경우는 제거하자
            else:
                ided.append(0)
        else:
            ided.append(0)
    df_fully['ID'] = ided

    drop_idx = df_fully.loc[df_fully.ID==-1].index
    print('peptide with 1 spectral count (remove this peptide) cnt : ', len(drop_idx))
    df_fully = df_fully.drop(drop_idx).reset_index(drop=True)

    drop_idx = np.where(df_fully.duplicated()==True)[0]
    df_fully = df_fully.drop(drop_idx).reset_index(drop=True)

    print('----ID/nonID cnt----\n', df_fully.ID.value_counts())  # massIVE-KB : half is semi_non_tryptic, half is fully_tryptic
    print('----final shape----\n', df_fully.shape)
    df_fully.head(1)
    return df_fully

In [18]:
df_fully = pep_detection_labelling()

del rows(including U, X amino acid) cnt : 160
peptide with 1 spectral count (remove this peptide) cnt :  113427
----ID/nonID cnt----
 0    1493757
1     480223
Name: ID, dtype: int64
----final shape----
 (1973980, 8)


In [ ]:
data = []
for prot, ptms, strips, c in df_kb[['PROTEIN', 'PTM_SEQ', 'STRIP_PEPTIDE', 'CHARGE']].values:
    for p in prot.split(';'):
        data.append([p, ptms, strips, c])
df_key = pd.DataFrame(data, columns=['protein', 'PTM', 'PEP', 'CHARGE']).drop_duplicates().reset_index(drop=True)
print(df_key.shape, len({(_, __) for _, __ in df_key[['protein', 'PEP']].values}))

In [102]:
df_fully_ptms = df_fully.loc[df_fully.ID==1].merge(df_key, on=['protein', 'PEP'], how='left')
df_fully_ptms = df_fully_ptms.dropna().reset_index(drop=True)
print(df_fully_ptms.shape)

cab_idx = [idx for idx, seq in zip(df_fully_ptms.index, df_fully_ptms.PTM.values) if '(Cab)' in seq]
ace_idx = [idx for idx, seq in zip(df_fully_ptms.index, df_fully_ptms.PTM.values) if '(Ace)' in seq]
aml_idx = [idx for idx, seq in zip(df_fully_ptms.index, df_fully_ptms.PTM.values) if '(Aml)' in seq]
drop_idx = set(cab_idx).union(set(ace_idx)).union(set(aml_idx))
df_fully_ptms=df_fully_ptms.drop(drop_idx, axis=0).reset_index(drop=True)
print(len(drop_idx))

print(df_fully_ptms.shape)
df_fully_ptms.head(2)

KeyboardInterrupt: 

In [29]:
detect_peps = df_fully_ptms.PTM.apply(lambda x: x.replace('(Cabm)', '').replace('(Dam)', '').replace('(Ox)', '')).values
detect_charge = df_fully_ptms.CHARGE.values
detect_pc = {(p, c) for p, c in zip(detect_peps, detect_charge)}
print(len(detect_pc))

In [34]:
df_key_realspec = pd.DataFrame([[p, c, '-'] for p, c in detect_pc], columns=['STRIP_PEPTIDE', 'CHARGE', 'DROP'])
df_kbreal = df_kb.merge(df_key_realspec, on=['STRIP_PEPTIDE', 'CHARGE'], how='inner')

cab_idx = [idx for idx, seq in zip(df_kbreal.index, df_kbreal.PTM_SEQ.values) if '(Cab)' in seq]
ace_idx = [idx for idx, seq in zip(df_kbreal.index, df_kbreal.PTM_SEQ.values) if '(Ace)' in seq]
aml_idx = [idx for idx, seq in zip(df_kbreal.index, df_kbreal.PTM_SEQ.values) if '(Aml)' in seq]
drop_idx = set(cab_idx).union(set(ace_idx)).union(set(aml_idx))
df_kbreal=df_kbreal.drop(drop_idx, axis=0).reset_index(drop=True)
print(df_kbreal.shape)

# 2019ACS_GuanModel Charge prediction

# 2019Nat_Prosit MS2 Prediction

In [5]:
def reshapeOneHot(X):
    X = np.dstack(X)
    X = np.swapaxes(X, 1, 2)
    X = np.swapaxes(X, 0, 1)
    return X

def one_hot_encode_peptide(peptide, MAX_LENGTH = 41):
    if len(peptide) > MAX_LENGTH:
        print('Peptide length is larger than maximal length of ', str(MAX_LENGTH))
        return None
    else:
        AA_vocabulary = 'KRPTNAQVSGILCMJHFYWEDBXO'#B: acetyl; O: Carbamyl; J: oxidized Met; X:pyro_glu
        no_not_used_aas = 1#U: not used

        one_hot_peptide = np.zeros((len(peptide), len(AA_vocabulary) - no_not_used_aas))

        for j in range(0, len(peptide)):
            try:
                aa = peptide[j]
                one_hot_peptide[j, AA_vocabulary.index(aa)] = 1
            except:
                pass
        
        no_front_paddings = int((MAX_LENGTH - len(peptide))/2)
        peptide_front_paddings = np.zeros((no_front_paddings, one_hot_peptide.shape[1]))

        no_back_paddings = MAX_LENGTH - len(peptide) - no_front_paddings
        peptide_back_paddings = np.zeros((no_back_paddings, one_hot_peptide.shape[1]))

        full_one_hot_peptide = np.vstack((peptide_front_paddings, one_hot_peptide, peptide_back_paddings))

        return peptide, full_one_hot_peptide
    
AA_vocabulary = 'KRPTNAQVSGILCMJHFYWEDBXOU'#B: acetyl; J: oxidized Met; X:pyro_glu
single2psi_lookup = {'B': '(Acetyl)-',
                     'J': 'M(Oxidation)',
                     'X': '(Gln->pyro-Glu)Q',
                     'C': 'C(Carbamidomethyl)'}

def one_hot_to_single_ptm(one_hot):
    seq = ''
    for row in one_hot:
        if row.sum() > 0:
            idx = row.argmax()
            seq += AA_vocabulary[idx]
    return seq



def single_ptm_to_psi(single_ptm_seq):
    psi_seq = single_ptm_seq
    for code in single2psi_lookup:
        psi_seq = psi_seq.replace(code, single2psi_lookup[code])
    return psi_seq

fn1 = '../refModel/2019MCP_Guan/LCMSMS_Pred_Supplemental_Material_section_S6/ChargeState/zfit_bidirLSTM2_masking_result.pickle'
with open(fn1, 'rb') as fid:
    pred_data = pickle.load(fid)

fn2 = '../refModel/2019MCP_Guan/LCMSMS_Pred_Supplemental_Material_section_S6/ChargeState/zfit_one_hot.pickle'
with open(fn2, 'rb') as fid:
    real_data = pickle.load(fid)

def plot_zfit(train_X, train_y, predicted_train_y, n):

    X = train_X[n]
    s_seq = one_hot_to_single_ptm(X)
    print(s_seq)
    psi_seq = single_ptm_to_psi(s_seq)
    print(psi_seq)
   
    y_exp = train_y[n]
    y_pred = predicted_train_y[n]
    pcc = pearsonr(y_exp, y_pred)

    fig, ax = plt.subplots()
    charges = range(1, len(y_exp) + 1)
    ax.stem(charges, y_exp, 'b', markerfmt=" ", label='Experimental')
    ax.stem(charges, -y_pred, 'g', markerfmt=" ", label='Predicted')
    plt.text(1.5, -0.35, 'pcc: %6.3f, %6.3g'%(pcc[0], pcc[1]))
    plt.xticks(charges)
    plt.xlabel('charge')
    plt.title(psi_seq)
    plt.show()

In [6]:
network_ori = tf.keras.models.load_model('../refModel/2019MCP_Guan/LCMSMS_Pred_Supplemental_Material_section_S6/ChargeState/zfit_bidirLSTM2_masking_model.h5')

print(network_ori.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, None, 23)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 512)         573440    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 2,280,965
Trainable params: 2,280,965
Non-trainable params: 0
_________________________________________________________________
None


* undetected peptides

In [43]:
undetect_peps = df_fully.loc[df_fully.ID==0].PEP.unique()
print(len(undetect_peps))

1422466


In [44]:
# additional prediction charge 6 peptide 
all_X_str = np.array(list(set(undetect_peps)))
all_onehot = dict()
for p in all_X_str:
    o = one_hot_encode_peptide(p)
    all_onehot[p] = o[1]
all_X = np.array(list(all_onehot.values()))
print(all_X.shape)

test=all_X[:1000]
kb_pred = network_ori.predict(test)

In [47]:
print(round(16.4*(1422466/1000)/3600, 2), 'hour estimated')

6.48 hour estimated


In [49]:
all_pred = network_ori.predict(all_X)

all_pred_log = np.array([np.array(list(map(lambda x: -math.log(x), _))) for _ in all_pred])
print(all_X_str.shape, all_X.shape, all_pred_log.shape)

peplist = []
for p, prob in zip(all_X_str, all_pred_log):
    idxes = np.where(prob<=6)[0]
    charges = idxes + 1  # casting
    for c in charges:
        peplist.append([p, 30, c])
df_peplist = pd.DataFrame(peplist, columns=['modified_sequence', 'collision_energy', 'precursor_charge'])
print(df_peplist.shape)

In [53]:
df_peplist.to_csv('undetect_peptides_MS2prediction_211115.csv', index=False)

* detected peptide

In [56]:
detect_peps = [_[0] for _ in detect_pc]

# additional prediction charge 6 peptide 
all_X_str = np.array(list(set(detect_peps)))
all_onehot = dict()
for p in all_X_str:
    o = one_hot_encode_peptide(p)
    all_onehot[p] = o[1]
all_X = np.array(list(all_onehot.values()))
print(all_X.shape)

test=all_X[:1000]
kb_pred = network_ori.predict(test)

In [73]:
print(round(25.3*(456572/1000)/3600, 2), 'hour estimated')

3.21 hour estimated


In [75]:
all_pred = network_ori.predict(all_X)

all_pred_log = np.array([np.array(list(map(lambda x: -math.log(x), _))) for _ in all_pred])
print(all_X_str.shape, all_X.shape, all_pred_log.shape)

peplist = []
for p, prob in zip(all_X_str, all_pred_log):
    idxes = np.where(prob<=6)[0]
    charges = idxes + 1  # casting
    for c in charges:
        peplist.append([p, 30, c])
df_peplist = pd.DataFrame(peplist, columns=['modified_sequence', 'collision_energy', 'precursor_charge'])
print(df_peplist.shape)

In [78]:
df_peplist.to_csv('detect_peptides_MS2prediction_211115.csv', index=False)

* modeling data

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
df_detect_peptide_train = pd.read_csv('../data/df_detect_peptide_train.csv')
test = pd.read_csv('../data/df_detect_peptide_test.csv')
train, val = train_test_split(df_detect_peptide_train, test_size=0.2, random_state=7)

df_modeling = pd.concat([train, val, test], axis=0)
print(len(df_modeling), len(df_modeling.PEP.unique()))
print('SET check', set(train.PEP.values).intersection(set(val.PEP.values)).intersection(set(test.PEP.values)))

813388 797400
SET check set()


In [7]:
# additional prediction charge 6 peptide 
all_X_str = np.array(df_modeling.PEP.values)
all_onehot = dict()
for p in all_X_str:
    o = one_hot_encode_peptide(p)
    all_onehot[p] = o[1]
all_X = np.array(list(all_onehot.values()))
print(len(all_X))

797400


In [8]:
all_pred = network_ori.predict(all_X)

In [9]:
all_pred_log = np.array([np.array(list(map(lambda x: -math.log(x), _))) for _ in all_pred])
print(all_X_str.shape, all_X.shape, all_pred_log.shape)

(813388,) (797400, 41, 23) (797400, 5)


In [42]:
peplist = []
for p, prob in zip(all_onehot.keys(), all_pred_log):
    idxes = np.where(prob<=6)[0]
    charges = idxes + 1  # casting
    for c in charges:
        peplist.append([p, 30, c])
df_peplist = pd.DataFrame(peplist, columns=['modified_sequence', 'collision_energy', 'precursor_charge'])

In [22]:
df_peplist.to_csv('df_modeling_211124.csv', index=False)

# MSGF+ Search for EDA
  - make mgf file

In [56]:
# Amino acids
MODIFICATION = {
    "CAM": 57.0214637236,  # Carbamidomethylation (CAM)
    "OX": 15.99491,  # Oxidation
}
AMINO_ACID = {
    "G": 57.021464,
    "R": 156.101111,
    "V": 99.068414,
    "P": 97.052764,
    "S": 87.032028,
    "U": 150.95363,
    "L": 113.084064,
    "M": 131.040485,
    "Q": 128.058578,
    "N": 114.042927,
    "Y": 163.063329,
    "E": 129.042593,
    "C": 103.009185,
    "F": 147.068414,
    "I": 113.084064,
    "A": 71.037114,
    "T": 101.047679,
    "W": 186.079313,
    "H": 137.058912,
    "D": 115.026943,
    "K": 128.094963,
}
AMINO_ACID["M(ox)"] = AMINO_ACID["M"] + MODIFICATION["OX"]
AMINO_ACID["C(cam)"] = AMINO_ACID["C"] + MODIFICATION["CAM"]

# Atomic elements
PROTON = 1.007276467
ELECTRON = 0.00054858
H = 1.007825035
C = 12.0
O = 15.99491463
N = 14.003074

# Tiny molecules
N_TERMINUS = H
C_TERMINUS = O + H
CO = C + O
CHO = C + H + O
NH2 = N + H * 2
H2O = H * 2 + O
NH3 = N + H * 3

def cal_MW(seq):
    R = sum([AMINO_ACID[aa] for aa in list(map(lambda x: x.replace('C', 'C(cam)'), seq))])
    return R + N_TERMINUS + C_TERMINUS

def cal_mz(seq, CHARGE):
    # we treat STRIP_PEPTIDE, so modification does not consider.
    MW = cal_MW(seq)
    return (MW + PROTON*CHARGE) / CHARGE

In [53]:
undetect_ms2 = pd.read_csv('undetect_peptides_MS2prediction_211025_result.msms', sep='\t')
print(undetect_ms2.shape)
undetect_ms2.head(2)

In [58]:
# peptide to spectrum dictionary (prediction)
start = time.time()

npep2spec_pred = dict()
for idx, (i, mz, p, c) in enumerate(undetect_ms2[['Intensities', 'Masses', 'Modified Sequence', 'Charge']].values):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')

    if (p, c) not in npep2spec_pred:
        npep2spec_pred[(p, c)] = dict()
    for i_, mz_ in zip(i.split(';'), mz.split(';')):
        npep2spec_pred[(p, c)][mz_]=i_

In [59]:
# nID mgf
start = time.time()

idf01 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac01.mgf', mode='w')
idf02 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac02.mgf', mode='w')
idf03 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac03.mgf', mode='w')
idf04 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac04.mgf', mode='w')
idf05 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac05.mgf', mode='w')
idf06 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac06.mgf', mode='w')
idf07 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac07.mgf', mode='w')
idf08 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac08.mgf', mode='w')
idf09 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac09.mgf', mode='w')
idf10 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac10.mgf', mode='w')
idf11 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac11.mgf', mode='w')
idf12 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac12.mgf', mode='w')
idf2frac = dict()
idf2frac[1]=idf01
idf2frac[2]=idf02
idf2frac[3]=idf03
idf2frac[4]=idf04
idf2frac[5]=idf05
idf2frac[6]=idf06
idf2frac[7]=idf07
idf2frac[8]=idf08
idf2frac[9]=idf09
idf2frac[10]=idf10
idf2frac[11]=idf11
idf2frac[12]=idf12
bins = [int(len(npep2spec_pred)/12 * i) for i in range(12)]
idx2fracidx = np.digitize(range(len(npep2spec_pred)), bins=bins)

for idx, ((p, c), mz2int) in enumerate(npep2spec_pred.items()):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')
        
    idf = idf2frac[idx2fracidx[idx]]
    idf.write('BEGIN IONS\n')
    idf.write('TITLE='+str(idx+1)+' File:\"uniprot_predMS2_frac{}.mgf", NativeID:\"controllerType=0 controllerNumber=1 scan='.format(str(idx+1).zfill(2))+str(idx+1)+'\"\n')
#     idf.write('RTINSECONDS=10\n')
    idf.write('PEPMASS='+str(cal_mz(p, c))+'\n')  # ' precursor_intensity'
    idf.write('CHARGE='+str(c)+'+\n')
    for mz, i in sorted(mz2int.items(), key=lambda x: x[0]):
        idf.write(str(mz)+' '+str(i)+'\n')
    idf.write('END IONS\n')
idf01.close()
idf02.close()
idf03.close()
idf04.close()
idf05.close()
idf06.close()
idf07.close()
idf08.close()
idf09.close()
idf10.close()
idf11.close()
idf12.close()

# massIVE-KB spectra

In [60]:
detect_ms2 = pd.read_csv('detect_peptides_MS2prediction_211025_result.msms', sep='\t')
print(detect_ms2.shape)
detect_ms2.head(2)

(764321, 5)


,Intensities,Masses,Matches,Modified Sequence,Charge
0,0.23332144;0.12056828;1.0;0.51005423;0.1888672...,175.118952167;288.203016167;375.235044167;474....,y1;y2;y3;y4;y5;y6;y7;y8;b1;b2;b3;b4;b5;b6;b7;y...,KKYEEEVSLR,3
1,0.1725661;0.33016136;0.10774748;0.2737293;0.60...,175.118952167;304.161545167;433.204138167;562....,y1;y2;y3;y4;y5;y6;y7;y8;b2;b3;b4;b5;b6;b7;b8;b...,AGVLAHLEEER,2


In [101]:
print(df_kbreal.shape)
df_kbreal.head(2)

,Unnamed: 0,PEPMASS,CHARGE,FILENAME,SEQ,PROTEIN,STRIP_PEPTIDE,SCORE,MZ,INTENSITY,SPECTRAL_CNT,PROTEIN_except_decoy,PROTEIN_except_else,PTM_SEQ,DROP
0,732863,406.208155,3,filtered_library_mgf_files/0c118258c5c04954969...,AAEEPSKVEEK,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK,5.561619,102.05590057373047;110.07201385498047;115.0874...,146919.9375;38942.06640625;651668.3125;25144.4...,85,sp|P29966|MARCS_HUMAN,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK,-
1,121794,653.582825,4,filtered_library_mgf_files/97f2d7cfb5a94249ad0...,AC+57.021QDVSVIIHTAC+57.021IIDVFGVTHR,sp|P14060|3BHS1_HUMAN,ACQDVSVIIHTACIIDVFGVTHR,9.530160,101.071044921875;115.08629608154297;120.080528...,3950.259521484375;7356.15966796875;18369.31445...,10,sp|P14060|3BHS1_HUMAN,sp|P14060|3BHS1_HUMAN,AC(Cabm)QDVSVIIHTAC(Cabm)IIDVFGVTHR,-


In [114]:
len({(pp, c) for pp, p, c in df_kbreal[['STRIP_PEPTIDE', 'PTM_SEQ', 'CHARGE']].values if '(Ox)' not in p and '(Dam)' not in p})

748884

In [63]:
# peptide to spectrum dictionary (prediction)
start = time.time()

pep2spec_pred = dict()
for idx, (i, mz, p, c) in enumerate(detect_ms2[['Intensities', 'Masses', 'Modified Sequence', 'Charge']].values):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')

    if (p, c) not in pep2spec_pred:
        pep2spec_pred[(p, c)] = dict()
    for i_, mz_ in zip(i.split(';'), mz.split(';')):
        pep2spec_pred[(p, c)][mz_]=i_

In [132]:
detect_ms2real = df_kbreal.loc[df_kbreal.PTM_SEQ.apply(lambda x: '(Ox)' not in x) & df_kbreal.PTM_SEQ.apply(lambda x: '(Dam)' not in x)]
detect_ms2real = detect_ms2real[['INTENSITY', 'MZ', 'STRIP_PEPTIDE', 'CHARGE']].rename({'INTENSITY':'Intensities',
                                                                                       'MZ':'Masses',
                                                                                       'STRIP_PEPTIDE':'Modified Sequence',
                                                                                       'CHARGE':'Charge'}, axis=1)

In [134]:
# peptide to spectrum dictionary
start = time.time()

pep2spec = dict()
for idx, (i, mz, p, c) in enumerate(detect_ms2real[['Intensities', 'Masses', 'Modified Sequence', 'Charge']].values):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')

    if (p, c) not in pep2spec:
        pep2spec[(p, c)] = dict()
    for i_, mz_ in zip(i.split(';'), mz.split(';')):
        pep2spec[(p, c)][mz_]=i_

In [65]:
import pickle
with open('../data/massIVE-KB/211025_stirp_pep2spec_pred.pickle', 'wb') as f:
    pickle.dump(pep2spec_pred, f, pickle.HIGHEST_PROTOCOL)
    
with open('../data/uniprot/211025_strip_npep2spec_pred.pickle', 'wb') as f:
    pickle.dump(npep2spec_pred, f, pickle.HIGHEST_PROTOCOL)

with open('../data/massIVE-KB/211108_stirp_pep2spec.pickle', 'wb') as f:
    pickle.dump(pep2spec, f, pickle.HIGHEST_PROTOCOL)

In [64]:
# ID mgf
start = time.time()

idf01 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac01.mgf', mode='w')
idf02 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac02.mgf', mode='w')
idf03 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac03.mgf', mode='w')
idf04 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac04.mgf', mode='w')
idf05 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac05.mgf', mode='w')
idf06 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac06.mgf', mode='w')
idf07 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac07.mgf', mode='w')
idf08 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac08.mgf', mode='w')
idf09 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac09.mgf', mode='w')
idf10 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac10.mgf', mode='w')
idf11 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac11.mgf', mode='w')
idf12 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac12.mgf', mode='w')
idf2frac = dict()
idf2frac[1]=idf01
idf2frac[2]=idf02
idf2frac[3]=idf03
idf2frac[4]=idf04
idf2frac[5]=idf05
idf2frac[6]=idf06
idf2frac[7]=idf07
idf2frac[8]=idf08
idf2frac[9]=idf09
idf2frac[10]=idf10
idf2frac[11]=idf11
idf2frac[12]=idf12
bins = [int(len(pep2spec_pred)/12 * i) for i in range(12)]
idx2fracidx = np.digitize(range(len(pep2spec_pred)), bins=bins)

for idx, ((p, c), mz2int) in enumerate(pep2spec_pred.items()):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')
    idf = idf2frac[idx2fracidx[idx]]
    idf.write('BEGIN IONS\n')
    idf.write('TITLE='+str(idx+1)+' File:\"massIVE-KB_predMS2_frac{}.mgf", NativeID:\"controllerType=0 controllerNumber=1 scan='.format(str(idx+1).zfill(2))+str(idx+1)+'\"\n')
    idf.write('PEPMASS='+str(cal_mz(p, c))+'\n')  # ' precursor_intensity'
    idf.write('CHARGE='+str(c)+'+\n')
    for mz, i in sorted(mz2int.items(), key=lambda x: x[0]):
        idf.write(str(mz)+' '+str(i)+'\n')
    idf.write('END IONS\n')

idf01.close()
idf02.close()
idf03.close()
idf04.close()
idf05.close()
idf06.close()
idf07.close()
idf08.close()
idf09.close()
idf10.close()
idf11.close()
idf12.close()

In [137]:
# ID mgf
start = time.time()

idf01 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac01.mgf', mode='w')
idf02 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac02.mgf', mode='w')
idf03 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac03.mgf', mode='w')
idf04 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac04.mgf', mode='w')
idf05 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac05.mgf', mode='w')
idf06 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac06.mgf', mode='w')
idf07 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac07.mgf', mode='w')
idf08 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac08.mgf', mode='w')
idf09 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac09.mgf', mode='w')
idf10 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac10.mgf', mode='w')
idf11 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac11.mgf', mode='w')
idf12 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac12.mgf', mode='w')
idf13 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac13.mgf', mode='w')
idf14 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac14.mgf', mode='w')
idf2frac = dict()
idf2frac[1]=idf01
idf2frac[2]=idf02
idf2frac[3]=idf03
idf2frac[4]=idf04
idf2frac[5]=idf05
idf2frac[6]=idf06
idf2frac[7]=idf07
idf2frac[8]=idf08
idf2frac[9]=idf09
idf2frac[10]=idf10
idf2frac[11]=idf11
idf2frac[12]=idf12
idf2frac[13]=idf13
idf2frac[14]=idf14
bins = [int(len(pep2spec)/14 * i) for i in range(14)]
idx2fracidx = np.digitize(range(len(pep2spec)), bins=bins)

for idx, ((p, c), mz2int) in enumerate(pep2spec.items()):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')
    idf = idf2frac[idx2fracidx[idx]]
    idf.write('BEGIN IONS\n')
    idf.write('TITLE='+str(idx+1)+' File:\"massIVE-KB_frac{}.mgf", NativeID:\"controllerType=0 controllerNumber=1 scan='.format(str(idx+1).zfill(2))+str(idx+1)+'\"\n')
    idf.write('PEPMASS='+str(cal_mz(p, c))+'\n')  # ' precursor_intensity'
    idf.write('CHARGE='+str(c)+'+\n')
    for mz, i in sorted(mz2int.items(), key=lambda x: x[0]):
        idf.write(str(mz)+' '+str(i)+'\n')
    idf.write('END IONS\n')

idf01.close()
idf02.close()
idf03.close()
idf04.close()
idf05.close()
idf06.close()
idf07.close()
idf08.close()
idf09.close()
idf10.close()
idf11.close()
idf12.close()
idf13.close()
idf14.close()